<a href="https://colab.research.google.com/github/kgautam2103/Zoom_analytics/blob/main/new_zoom_chat_extract_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import sys
import re
import pandas as pd
import numpy as np
import traceback
from io import StringIO

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WhACIU6F7i9VwDxXoqzx6GEwutQI5HpIYl4v6H49-7Q/edit#gid=0')

input_sheet = wb.worksheet('sahajyogis')
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['id','name','email'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])
ref_name_list = list(filter(None, inputdata_df['name'].tolist()))
ref_name_list = [i.strip().lower() for i in ref_name_list]

from google.colab import files
chat_upload = files.upload()
filepath = input('enter the file path from the content folder: ')

lines = open(filepath).read().splitlines()

tags = []
messages = []


while lines:
    tags.append(lines.pop(0))
    message = ""
    while (lines and lines[0].startswith('\t')):
        if len(message) == 0:
            message += lines.pop(0).lstrip("\t")
        else:
            message += ', ' + lines.pop(0).lstrip("\t")
    messages.append(message)

name_list=[]
for name_detail in tags:
  from_index = name_detail.find(" From ")
  to_index = name_detail.find(" To ")
  if to_index == -1:
      name = name_detail[from_index + 6:]
  else:
      name = name_detail[from_index + 6:to_index]
  name_list.append(name.strip())
  #print(from_index,to_index,name)

email_list = []

pattern = r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+'
emails = [re.search(pattern, msg) for msg in messages]
for i in range(len(emails)):
    if emails[i]:
        emails[i] = emails[i].group(0)
email_list = ["" if not email else email for email in emails]

extract_df = pd.DataFrame(list(zip(name_list, email_list,messages)),
               columns =['name','email' ,'chat'])
#final_df_email = extract_df.groupby("name").apply(lambda x: x["email"].tolist())
final_df_email = extract_df.groupby("name").apply(lambda x: [x["email"].tolist(), x["chat"].tolist()]).apply(pd.Series)
#final_df_email.columns=["name","email"]

#concat_df = pd.concat([final_df_email[0].reset_index(),final_df_email[1].reset_index()],axis=1)

merge_df = pd.merge(final_df_email[0].reset_index(),final_df_email[1].reset_index(),on="name")

final_df = merge_df.rename(columns={0:'email',1:'chat'}, inplace=False)

final_df['email'] = [(','.join(list(set(rx)))).strip(',')for rx in final_df['email']]

final_df['chat'] = [(', '.join(rx)).strip(',')for rx in final_df['chat']]

final_df['name'] = final_df['name'].apply(lambda x: re.sub('^sy[\.\s_-]*','',x,flags=re.IGNORECASE))\
  .apply(lambda x: re.sub('[\.\s_-]*sy$','',x,flags=re.IGNORECASE))

final_df = final_df[~(final_df.name).str.lower().isin(ref_name_list)]

final_df.to_csv('ZoomChat_Extract.csv')
files.download("ZoomChat_Extract.csv")

#final_df

Saving meeting_saved_chat.txt to meeting_saved_chat.txt
enter the file path from the content folder: /content/meeting_saved_chat.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# New section

# New section